## Notebook for testing code snippets

In [ ]:
#testing uncertainty calculation on area estimates following Olofsson et al. 2013 Making better use of accuracy data in land change studies.

In [1]:
import numpy as np
import pandas as pd

In [139]:
confusion_matrix = 'results/nmdb_plots/uncertainty/test_uncertainty.csv'
year = '1989-90'

In [112]:
a = pd.read_csv(confusion_matrix, index_col='Class')
a['map_area'] = a['map_area']*(5*5) / 10000
a

,irrigated,non_irrigated,total,map_area,Wi
Class,,,,,
irrigated,245,5,250,5.353589e+04,0.020548
non_irrigated,1,249,250,2.551910e+06,0.979452
total,246,254,500,2.605446e+06,1.000000


In [134]:
def areaUncertainty(df):
    stratified_estimate = a['Wi'][0]*(a['irrigated'][0]/a['total'][0])+a['Wi'][1]*(a['irrigated'][1]/a['total'][1])
    adjusted_area = a['map_area'][-1] * stratified_estimate
    #standard error
    i = round(a['Wi'][0], 6)**2 * (a['irrigated'][0]/a['total'][0] * (1-a['irrigated'][0]/a['total'][0]) / int(a['total'][0]-1))
    j = round(a['Wi'][1], 6)**2 * (a['irrigated'][1]/a['total'][1] * (1-a['irrigated'][1]/a['total'][1]) / int(a['total'][1]-1))
    standard_error = (i + j)**(0.5)
    SE_of_adjustedArea = a['map_area'][-1] * standard_error
    CI = 2 * SE_of_adjustedArea
    return adjusted_area, CI
    

In [136]:
adjusted_area, CI = areaUncertainty(a)

In [141]:
print("Irrigated area estimate for " +year +", with 95% confidence interval: " + str(round(adjusted_area, 2)) + " +- " + str(round(CI,2)) + " Ha")

Irrigated area estimate for 1989-90, with 95% confidence interval: 62672.81 +- 20437.36 Ha


In [ ]:
# simplifying polygons

In [ ]:
import sys
import os
import time
import copy
import geopandas as gpd

from shapely.geometry import asShape
from shapely.geometry import MultiLineString
from shapely.geometry import asLineString
from shapely.wkt import dumps
#from pprint import pprint

import shapefile
import numpy as np

sys.path.append('src/')
import bezier
import bendsimplify

In [ ]:
shape = "results/SICA/nmdb_Summer1998_99_Irrigated_OEHandLS_masked.shp"

In [ ]:
#nothihng seems to happen...
bendsimplify.bend_simplify("results/nmdb_Summer1998_99_Irrigated_OEHandLS_masked_objectID.shp")

In [ ]:
# This works but is not topologically aware so overlaps/slivers occur (if not
# for important analysis then could use overlay to remove overlaps)
gdf = gpd.read_file(shape)
gdf_simplified = gdf.copy()
gdf_simplified["geometry"] = gdf.geometry.simplify(tolerance=50,preserve_topology=True)
gdf.to_file("results/test_simplify_1998_99_tolerance50_shapely.shp")